In [2]:
import sys
from os.path import dirname
sys.path.append("/home/jason/sproj/donkeycar")

import donkeycar as dk
#import parts
from donkeycar.parts.camera import PiCamera
from donkeycar.parts.transform import Lambda
from donkeycar.parts.keras import KerasCategorical
from donkeycar.parts.actuator import PCA9685, PWMSteering, PWMThrottle
from donkeycar.parts.datastore import TubHandler, TubGroup
from donkeycar.parts.controller import LocalWebController, JoystickController

using donkey v2.2.1 ...


/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [7]:
from keras.layers import Input, Dense, merge
from keras.models import Model, Sequential
from keras.layers import Convolution2D, MaxPooling2D, Reshape, BatchNormalization,Cropping2D
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.layers.wrappers import TimeDistributed as TD
from keras.layers import LSTM

image_shape=(120,160,3)
seq_length=3
num_outputs=2
hidden_size = 1152

img_seq_shape = (seq_length,) + image_shape   
img_in = Input(batch_shape = img_seq_shape, name='img_in')

img_in = Input(shape=(120, 160, 3), name='img_in')                    
model = Sequential()
model.add(TD(Cropping2D(cropping=((60,0), (0,0))), input_shape=img_seq_shape )) #trim 60 pixels off top
model.add(TD(Convolution2D(24, (5,5), strides=(2,2), activation='relu')))
model.add(TD(Convolution2D(32, (5,5), strides=(2,2), activation='relu')))
model.add(TD(Convolution2D(32, (3,3), strides=(2,2), activation='relu')))
model.add(TD(Convolution2D(32, (3,3), strides=(1,1), activation='relu')))
model.add(TD(MaxPooling2D(pool_size=(2, 2))))
model.add(TD(Flatten(name='flattened')))
model.add(TD(Dense(100, activation='relu')))
model.add(TD(Dropout(.1)))

model.add(LSTM(128, return_sequences=True, name="LSTM_seq"))
model.add(Dropout(.1))
model.add(LSTM(128, return_sequences=False, name="LSTM_out"))
model.add(Dropout(.1))
model.add(Dense(128, activation='relu'))
model.add(Dropout(.1))
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(num_outputs, activation='linear', name='model_outputs'))

model.compile(optimizer="adam",
                  loss='mse')

In [9]:
img_in.shape

TensorShape([Dimension(None), Dimension(120), Dimension(160), Dimension(3)])

In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_19 (TimeDis (None, 3, 60, 160, 3)     0         
_________________________________________________________________
time_distributed_20 (TimeDis (None, 3, 28, 78, 24)     1824      
_________________________________________________________________
time_distributed_21 (TimeDis (None, 3, 12, 37, 32)     19232     
_________________________________________________________________
time_distributed_22 (TimeDis (None, 3, 5, 18, 32)      9248      
_________________________________________________________________
time_distributed_23 (TimeDis (None, 3, 3, 16, 32)      9248      
_________________________________________________________________
time_distributed_24 (TimeDis (None, 3, 1, 8, 32)       0         
_________________________________________________________________
time_distributed_25 (TimeDis (None, 3, 256)            0         
__________